<a href="https://colab.research.google.com/github/markchiang/FacialScores/blob/main/FacialScores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone https://github.com/markchiang/FacialScores.git
!cp -R FacialScores/?? ./

fatal: destination path 'FacialScores' already exists and is not an empty directory.


In [15]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Define function to load images and labels from folders
def load_labeled_dataset():
    images = []
    labels = []
    for age in range(20, 80, 10):
        folder_path = f'{age}'
        image_filenames = os.listdir(folder_path)
        for image_filename in image_filenames:
          if os.path.splitext(image_filename)[1].lower() in ('.jpg', '.jpeg', '.png', '.bmp'):
            image_path = os.path.join(folder_path, image_filename)
            image = cv2.imread(image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (128, 128), interpolation=cv2.INTER_AREA)
            images.append(resized)
            labels.append(age)
    return np.array(images), np.array(labels)

# Preprocess images for model input
def preprocess_images(images):
    normalized = images / 255.0
    batched = np.expand_dims(normalized, axis=-1)
    return batched

# Load dataset of labeled images
images, labels = load_labeled_dataset()

# Preprocess images for model input
preprocessed_images = preprocess_images(images)

# Define model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compile model with loss function and optimizer
model.compile(loss='mse', optimizer=optimizers.Adam(lr=0.001))

# Train model on labeled dataset
model.fit(preprocessed_images, labels, epochs=100, batch_size=32)

# Save model for future use
model.save('facial_condition_model.h5')


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 2311.0676
Epoch 2/100
1/1 [==============================] - 0s 396ms/step - loss: 1997.6549
Epoch 3/100
1/1 [==============================] - 0s 404ms/step - loss: 1473.9037
Epoch 4/100
1/1 [==============================] - 0s 416ms/step - loss: 758.4170
Epoch 5/100
1/1 [==============================] - 0s 401ms/step - loss: 341.4496
Epoch 6/100
1/1 [==============================] - 0s 381ms/step - loss: 1087.9471
Epoch 7/100
1/1 [==============================] - 0s 384ms/step - loss: 702.6841
Epoch 8/100
1/1 [==============================] - 0s 391ms/step - loss: 360.7782
Epoch 9/100
1/1 [==============================] - 0s 380ms/step - loss: 356.1537
Epoch 10/100
1/1 [==============================] - 0s 371ms/step - loss: 476.1395
Epoch 11/100
1/1 [==============================] - 0s 382ms/step - loss: 566.4542
Epoch 12/100
1/1 [==============================] - 0s 380ms/step - loss: 586.3585
Epoch 13/100

In [16]:

# Load pre-trained model
model = tf.keras.models.load_model('facial_condition_model.h5')

# Function to preprocess image for model input
def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize image to 128x128 pixels
    resized = cv2.resize(gray, (128, 128), interpolation=cv2.INTER_AREA)
    # Convert image to float32 and normalize pixel values
    normalized = np.float32(resized) / 255.0
    # Add batch dimension to image
    batched = np.expand_dims(normalized, axis=0)
    return batched

# Function to calculate facial condition scores
def calculate_scores(image):
    # Preprocess image for model input
    preprocessed_image = preprocess_image(image)
    # Use pre-trained model to predict facial condition scores
    score = model.predict(preprocessed_image)[0][0]
    # Calculate wrinkle score
    wrinkle_score = 1 - (1 / (1 + np.exp(-5*(score-50)/50)))
    # Calculate laxity score
    laxity_score = np.tanh(score / 10)
    # Calculate facial age
    facial_age = int(np.round(score / 10) * 10)
    # Calculate final score
    final_score = np.mean([wrinkle_score, laxity_score, facial_age])
    return wrinkle_score, laxity_score, facial_age, final_score


In [17]:

# Example usage
image = cv2.imread('20/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

image = cv2.imread('30/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

image = cv2.imread('40/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

image = cv2.imread('50/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

image = cv2.imread('60/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)


1/1 [==============================] - 0s 97ms/step
Wrinkle score: 0.9538567752952879
Laxity score: 0.961938059393008
Facial age: 20
Final score: 7.3052649448960985
1/1 [==============================] - 0s 35ms/step
Wrinkle score: 0.8832898138462825
Laxity score: 0.9948126420927021
Facial age: 30
Final score: 10.626034151979662
1/1 [==============================] - 0s 38ms/step
Wrinkle score: 0.7382559764594188
Laxity score: 0.9992779143006791
Facial age: 40
Final score: 13.912511296920032
1/1 [==============================] - 0s 30ms/step
Wrinkle score: 0.48782779915002616
Laxity score: 0.9999176301310339
Facial age: 50
Final score: 17.162581809760354
1/1 [==============================] - 0s 132ms/step
Wrinkle score: 0.2808609382861864
Laxity score: 0.9999861503405146
Facial age: 60
Final score: 20.426949029542232


In [18]:
# Function to calculate facial condition scores
def calculate_scores2(image):
    # Preprocess image for model input
    preprocessed_image = preprocess_image(image)
    # Use pre-trained model to predict facial condition scores
    score = model.predict(preprocessed_image)[0][0]
    # Calculate wrinkle score
    wrinkle_score = 1 - (1 / (1 + np.exp(-5*(score-50)/50)))
    # Calculate laxity score
    laxity_score = np.tanh(score / 10)
    # Calculate facial age
    facial_age = int(np.round(score / 10) * 10)
    # Calculate final score
    final_score = np.mean([wrinkle_score, laxity_score, facial_age])
    return wrinkle_score, laxity_score, facial_age, final_score


In [20]:
import cv2
import urllib
import numpy as np

req = urllib.request.urlopen('https://media.licdn.com/dms/image/C5603AQEiW9vZPeekiw/profile-displayphoto-shrink_800_800/0/1516317900912?e=2147483647&v=beta&t=Rs0LpxDfsrHmQFzD0w1vRpCRln0q-sPWrAY9lgiqYeU')
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
image = cv2.imdecode(arr, -1) # 'Load it as it is'
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores2(image)
print('---before---')
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

req = urllib.request.urlopen('https://scontent.ftpe8-1.fna.fbcdn.net/v/t1.6435-9/195542178_605699050027_1477839146608127555_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=dbeb18&_nc_ohc=fUhwWIuKSb0AX_Irg8T&_nc_ht=scontent.ftpe8-1.fna&oh=00_AfAg2ep5yEM_fzNubC3hS7P22hGWq_rR0WluvTBRkrSGvA&oe=6465EF97')
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
image = cv2.imdecode(arr, -1) # 'Load it as it is'
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores2(image)
print('---after---')
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

1/1 [==============================] - 0s 29ms/step
---before---
Wrinkle score: 0.9305396907894703
Laxity score: 0.9838356867202137
Facial age: 20
Final score: 7.304791792503228
1/1 [==============================] - 0s 29ms/step
---after---
Wrinkle score: 0.5072287290798501
Laxity score: 0.9999037986395953
Facial age: 50
Final score: 17.169044175906482
